In [1]:
import sys
sys.path.append('/home/fedor/models')
sys.path.append('/home/fedor/models/slim')

In [2]:
import os

import functools
import tensorflow as tf
from google.protobuf import text_format

from object_detection import trainer
from object_detection import evaluator
from object_detection.builders import model_builder
from object_detection.builders import input_reader_builder
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util

In [3]:
config_path = 'ssd_mobilenet_v1_nexar.config'
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(config_path, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)

model_config = pipeline_config.model
train_config = pipeline_config.train_config
input_config = pipeline_config.train_input_reader

In [4]:
model_fn = functools.partial(model_builder.build, model_config=model_config, is_training=True)
create_input_dict_fn = functools.partial(input_reader_builder.build, input_config)

In [5]:
!nvidia-smi

Wed Aug 30 19:11:39 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 0000:01:00.0     Off |                  N/A |
| 27%   44C    P8    19W / 250W |   6940MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     Off  | 0000:02:00.0     Off |                  N/A |
| 24%   41C    P8    18W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
trainer.train(create_input_dict_fn, model_fn, train_config,
              master='', task=0, num_clones=1,
              worker_replicas=1, clone_on_cpu=False, ps_tasks=0,
              worker_job_name='lonely_worker', is_chief=True, train_dir='/home/fedor/CycleGAN-habrahabr/code/logs/train')

INFO:tensorflow:global step 128886: loss = 2.0024 (0.419 sec/step)
INFO:tensorflow:global step 128887: loss = 2.0720 (0.382 sec/step)
INFO:tensorflow:global step 128888: loss = 1.6999 (0.412 sec/step)
INFO:tensorflow:global step 128889: loss = 2.2124 (0.408 sec/step)
INFO:tensorflow:global step 128890: loss = 2.5269 (0.405 sec/step)
INFO:tensorflow:global step 128891: loss = 1.8767 (0.414 sec/step)
INFO:tensorflow:global step 128892: loss = 2.6641 (0.444 sec/step)
INFO:tensorflow:global step 128893: loss = 1.9125 (0.433 sec/step)
INFO:tensorflow:global step 128894: loss = 2.1892 (0.425 sec/step)
INFO:tensorflow:global step 128895: loss = 1.7411 (0.358 sec/step)
INFO:tensorflow:global step 128896: loss = 1.8944 (0.414 sec/step)
INFO:tensorflow:global step 128897: loss = 1.8669 (0.409 sec/step)
INFO:tensorflow:global step 128898: loss = 2.1893 (0.414 sec/step)
INFO:tensorflow:global step 128899: loss = 2.2912 (0.417 sec/step)
INFO:tensorflow:global step 128900: loss = 2.4158 (0.416 sec/s

INFO:tensorflow:global step 129007: loss = 2.0051 (0.412 sec/step)
INFO:tensorflow:global step 129008: loss = 1.9899 (0.427 sec/step)
INFO:tensorflow:global step 129009: loss = 1.8484 (0.423 sec/step)
INFO:tensorflow:global step 129010: loss = 2.1628 (0.428 sec/step)
INFO:tensorflow:global step 129011: loss = 2.5031 (0.412 sec/step)
INFO:tensorflow:global step 129012: loss = 1.7157 (0.429 sec/step)
INFO:tensorflow:global step 129013: loss = 2.9176 (0.380 sec/step)
INFO:tensorflow:global step 129014: loss = 2.5658 (0.435 sec/step)
INFO:tensorflow:global step 129015: loss = 2.0209 (0.418 sec/step)
INFO:tensorflow:global step 129016: loss = 1.9671 (0.419 sec/step)
INFO:tensorflow:global step 129017: loss = 1.9309 (0.421 sec/step)
INFO:tensorflow:global step 129018: loss = 1.5885 (0.415 sec/step)
INFO:tensorflow:global step 129019: loss = 1.9744 (0.389 sec/step)
INFO:tensorflow:global step 129020: loss = 2.0426 (0.412 sec/step)
INFO:tensorflow:global step 129021: loss = 2.8279 (0.407 sec/s

INFO:tensorflow:global step 129130: loss = 2.1935 (0.423 sec/step)
INFO:tensorflow:global step 129131: loss = 2.3593 (0.402 sec/step)
INFO:tensorflow:global step 129132: loss = 2.8198 (0.411 sec/step)
INFO:tensorflow:global step 129133: loss = 1.7645 (0.414 sec/step)
INFO:tensorflow:global step 129134: loss = 2.0329 (0.386 sec/step)
INFO:tensorflow:global step 129135: loss = 1.7105 (0.380 sec/step)
INFO:tensorflow:global step 129136: loss = 2.0059 (0.391 sec/step)
INFO:tensorflow:global step 129137: loss = 2.2368 (0.412 sec/step)
INFO:tensorflow:global step 129138: loss = 1.9695 (0.403 sec/step)
INFO:tensorflow:global step 129139: loss = 2.3816 (0.417 sec/step)
INFO:tensorflow:global step 129140: loss = 2.1742 (0.409 sec/step)
INFO:tensorflow:global step 129141: loss = 2.0411 (0.412 sec/step)
INFO:tensorflow:global step 129142: loss = 2.3549 (0.418 sec/step)
INFO:tensorflow:global step 129143: loss = 2.2521 (0.406 sec/step)
INFO:tensorflow:global step 129144: loss = 2.3561 (0.422 sec/s

INFO:tensorflow:global step 129253: loss = 2.1106 (0.412 sec/step)
INFO:tensorflow:global step 129254: loss = 2.2025 (0.411 sec/step)
INFO:tensorflow:global step 129255: loss = 2.2941 (0.409 sec/step)
INFO:tensorflow:global step 129256: loss = 1.5958 (0.678 sec/step)
INFO:tensorflow:Recording summary at step 129256.
INFO:tensorflow:global step 129257: loss = 2.3123 (0.449 sec/step)
INFO:tensorflow:global step 129258: loss = 2.0201 (0.438 sec/step)
INFO:tensorflow:global step 129259: loss = 2.9265 (0.405 sec/step)
INFO:tensorflow:global step 129260: loss = 2.6149 (0.413 sec/step)
INFO:tensorflow:global step 129261: loss = 2.3963 (0.407 sec/step)
INFO:tensorflow:global step 129262: loss = 2.3685 (0.420 sec/step)
INFO:tensorflow:global step 129263: loss = 2.4726 (0.410 sec/step)
INFO:tensorflow:global step 129264: loss = 1.9620 (0.402 sec/step)
INFO:tensorflow:global step 129265: loss = 1.5468 (0.413 sec/step)
INFO:tensorflow:global step 129266: loss = 2.2897 (0.418 sec/step)
INFO:tensorf

INFO:tensorflow:global step 129375: loss = 1.9320 (0.427 sec/step)
INFO:tensorflow:global step 129376: loss = 1.8410 (0.406 sec/step)
INFO:tensorflow:global step 129377: loss = 1.7247 (0.416 sec/step)
INFO:tensorflow:global step 129378: loss = 2.3951 (0.429 sec/step)
INFO:tensorflow:global step 129379: loss = 1.5545 (0.416 sec/step)
INFO:tensorflow:global step 129380: loss = 2.6776 (0.419 sec/step)
INFO:tensorflow:global step 129381: loss = 1.7487 (0.413 sec/step)
INFO:tensorflow:global step 129382: loss = 2.3635 (0.431 sec/step)
INFO:tensorflow:global step 129383: loss = 2.2147 (0.412 sec/step)
INFO:tensorflow:global step 129384: loss = 2.1380 (0.420 sec/step)
INFO:tensorflow:global step 129385: loss = 2.5566 (0.404 sec/step)
INFO:tensorflow:global step 129386: loss = 2.5585 (0.381 sec/step)
INFO:tensorflow:global step 129387: loss = 1.7860 (0.416 sec/step)
INFO:tensorflow:global step 129388: loss = 3.3336 (0.410 sec/step)
INFO:tensorflow:global step 129389: loss = 2.5863 (0.414 sec/s

...

INFO:tensorflow:global step 134152: loss = 1.9626 (0.414 sec/step)
INFO:tensorflow:global step 134153: loss = 1.7807 (0.416 sec/step)
INFO:tensorflow:global step 134154: loss = 1.7489 (0.421 sec/step)
INFO:tensorflow:global step 134155: loss = 2.5700 (0.415 sec/step)
INFO:tensorflow:global step 134156: loss = 2.3350 (0.441 sec/step)
INFO:tensorflow:global step 134157: loss = 1.8215 (0.400 sec/step)
INFO:tensorflow:global step 134158: loss = 2.6220 (0.411 sec/step)
INFO:tensorflow:global step 134159: loss = 2.2767 (0.401 sec/step)
INFO:tensorflow:global step 134160: loss = 1.4352 (0.413 sec/step)
INFO:tensorflow:global step 134161: loss = 2.1129 (0.429 sec/step)
INFO:tensorflow:global step 134162: loss = 2.0429 (0.416 sec/step)
INFO:tensorflow:global step 134163: loss = 1.8125 (0.400 sec/step)
INFO:tensorflow:global step 134164: loss = 2.4030 (0.414 sec/step)
INFO:tensorflow:global step 134165: loss = 2.2916 (0.415 sec/step)
INFO:tensorflow:global step 134166: loss = 1.7585 (0.414 sec/s

INFO:tensorflow:global step 134275: loss = 2.1190 (0.430 sec/step)
INFO:tensorflow:global step 134276: loss = 2.1100 (0.410 sec/step)
INFO:tensorflow:global step 134277: loss = 1.8249 (0.410 sec/step)
INFO:tensorflow:global step 134278: loss = 1.8400 (0.433 sec/step)
INFO:tensorflow:global step 134279: loss = 1.8027 (0.424 sec/step)
INFO:tensorflow:global step 134280: loss = 1.9825 (0.424 sec/step)
INFO:tensorflow:global step 134281: loss = 1.3203 (0.421 sec/step)
INFO:tensorflow:global step 134282: loss = 2.1738 (0.418 sec/step)
INFO:tensorflow:global step 134283: loss = 2.1926 (0.411 sec/step)
INFO:tensorflow:global step 134284: loss = 2.0831 (0.415 sec/step)
INFO:tensorflow:global step 134285: loss = 2.0848 (0.433 sec/step)
INFO:tensorflow:global step 134286: loss = 1.7023 (0.416 sec/step)
INFO:tensorflow:global step 134287: loss = 1.5513 (0.412 sec/step)
INFO:tensorflow:global step 134288: loss = 2.4471 (0.367 sec/step)
INFO:tensorflow:global step 134289: loss = 2.2842 (0.418 sec/s

INFO:tensorflow:global step 134398: loss = 1.7896 (0.425 sec/step)
INFO:tensorflow:global step 134399: loss = 2.3897 (0.418 sec/step)
INFO:tensorflow:global step 134400: loss = 1.6697 (0.409 sec/step)
INFO:tensorflow:global step 134401: loss = 1.7824 (0.425 sec/step)
INFO:tensorflow:global step 134402: loss = 1.7222 (0.409 sec/step)
INFO:tensorflow:global step 134403: loss = 2.2481 (0.399 sec/step)
INFO:tensorflow:global step 134404: loss = 1.6386 (0.419 sec/step)
INFO:tensorflow:global step 134405: loss = 1.5888 (0.426 sec/step)
INFO:tensorflow:global step 134406: loss = 1.9280 (0.410 sec/step)
INFO:tensorflow:global step 134407: loss = 1.8391 (0.554 sec/step)
INFO:tensorflow:Recording summary at step 134407.
INFO:tensorflow:global step 134408: loss = 2.3502 (0.471 sec/step)
INFO:tensorflow:global step 134409: loss = 2.4555 (0.407 sec/step)
INFO:tensorflow:global step 134410: loss = 2.0809 (0.425 sec/step)
INFO:tensorflow:global step 134411: loss = 1.9027 (0.412 sec/step)
INFO:tensorf

INFO:tensorflow:global step 134520: loss = 2.1173 (0.412 sec/step)
INFO:tensorflow:global step 134521: loss = 1.7672 (0.414 sec/step)
INFO:tensorflow:global step 134522: loss = 2.2685 (0.430 sec/step)
INFO:tensorflow:global step 134523: loss = 2.0839 (0.427 sec/step)
INFO:tensorflow:global step 134524: loss = 2.0261 (0.409 sec/step)
INFO:tensorflow:global step 134525: loss = 2.5723 (0.397 sec/step)
INFO:tensorflow:global step 134526: loss = 1.9494 (0.419 sec/step)
INFO:tensorflow:global step 134527: loss = 2.2348 (0.427 sec/step)
INFO:tensorflow:global step 134528: loss = 1.8023 (0.402 sec/step)
INFO:tensorflow:global step 134529: loss = 1.9452 (0.415 sec/step)
INFO:tensorflow:global step 134530: loss = 1.8657 (0.423 sec/step)
INFO:tensorflow:global step 134531: loss = 2.3105 (0.420 sec/step)
INFO:tensorflow:global step 134532: loss = 1.9841 (0.425 sec/step)
INFO:tensorflow:global step 134533: loss = 2.0516 (0.411 sec/step)
INFO:tensorflow:global step 134534: loss = 1.9804 (0.423 sec/s

KeyboardInterrupt: 